In [147]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [148]:
steps = pd.read_csv("data_prep_recipe.csv")

In [149]:
df = pd.read_csv("movie_metadata.csv")

In [150]:
median_cols = steps.loc[steps["median"]==1,"column"].tolist()

In [151]:
imp_median = SimpleImputer(strategy='median')
df[median_cols] = imp_median.fit_transform(df[median_cols])

In [152]:
const_cols = steps.loc[steps["constant"]!="0",["column","constant"]].reset_index(drop=True)

In [153]:
for i in range(0,const_cols.shape[0]):
    imp_median = SimpleImputer(strategy='constant',fill_value=const_cols.loc[i,"constant"])
    df[const_cols.loc[i,"column"]] = imp_median.fit_transform(df[const_cols.loc[i,"column"]].values.reshape(-1, 1))

In [154]:
drop_cols = steps.loc[steps["drop"]==1,"column"].tolist()

In [155]:
df = df.drop(drop_cols,axis=1)

In [156]:
dummy_cols = steps.loc[steps["dummy"]==1,"column"].tolist()

In [157]:
# Create dummy variables
cat_df = df[dummy_cols]
df.drop(dummy_cols,axis=1,inplace=True)


In [158]:
from sklearn.preprocessing import MultiLabelBinarizer

for dvar in dummy_cols:
    s =cat_df[dvar].str.split(pat="|")
    mlb = MultiLabelBinarizer()
    df= pd.concat([df,pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)],axis=1)


In [159]:
df.isna().sum()

num_critic_for_reviews     0
duration                   0
director_facebook_likes    0
actor_3_facebook_likes     0
actor_1_facebook_likes     0
                          ..
TV-PG                      0
TV-Y                       0
TV-Y7                      0
Unrated                    0
X                          0
Length: 172, dtype: int64

In [160]:
# All missing values removed, saving to file
df.to_csv("cleaned_imdb.csv",index=False)